<a href="https://colab.research.google.com/github/pgasp/starburst-python/blob/main/tpch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring PyStarburst with Starburst and the TPC-H dataset

## Getting started

### Load the Dataframe (DF) API

Pull up https://pystarburst.eng.starburstdata.net/ in a browser window.

## Explore via code examples

Lets go!

In [1]:
#
# Install the library
#

%pip install pystarburst

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: zstandard
    Found existing installation: zstandard 0.24.0
    Uninstalling zstandard-0.24.0:
      Successfully uninstalled zstandard-0.24.0


In [2]:
#
# Define connection properties
#  get the host and other information from the cluster list
#

import getpass

host = "ai-workshop.enablement.starburstdata.net"
username = "pascal.gasp"
password = getpass.getpass("Password")


Password··········


In [3]:
#
# Import dependencies
#

from pystarburst import Session
from pystarburst import functions as F
from pystarburst.functions import *
from pystarburst.window import Window as W

import trino

session_properties = {
    "host":host,
    "port": 443,
    # Needed for https secured clusters
    "http_scheme": "https",
    # Setup authentication through login or password or any other supported authentication methods
    # See docs: https://github.com/trinodb/trino-python-client#authentication-mechanisms
    "auth": trino.auth.BasicAuthentication(username, password)
}

session = Session.builder.configs(session_properties).create()

In [4]:
#
# Validate connectivity to the cluster
#

session.sql("select 1 as b").collect()

[Row(b=1)]

In [5]:
#
# Ensure we have access to the TPC-H dataset by listing the tables in the tiny schema
#  https://pystarburst.eng.starburstdata.net/session.html#pystarburst.session.Session.sql
#

session.sql("show tables from tpch.tiny").collect()

[Row(Table='customer'),
 Row(Table='lineitem'),
 Row(Table='nation'),
 Row(Table='orders'),
 Row(Table='part'),
 Row(Table='partsupp'),
 Row(Table='region'),
 Row(Table='supplier')]

In [6]:
#
# What columns make up the lineitem table
#  https://pystarburst.eng.starburstdata.net/session.html#pystarburst.session.Session.table
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.schema
#

# Create a Dataframe for the lineitem table
tli = session.table("tpch.tiny.lineitem")

# Show the columns
print(tli.schema)

StructType([StructField('orderkey', LongType(), nullable=True), StructField('partkey', LongType(), nullable=True), StructField('suppkey', LongType(), nullable=True), StructField('linenumber', IntegerType(), nullable=True), StructField('quantity', DoubleType(), nullable=True), StructField('extendedprice', DoubleType(), nullable=True), StructField('discount', DoubleType(), nullable=True), StructField('tax', DoubleType(), nullable=True), StructField('returnflag', StringType(), nullable=True), StructField('linestatus', StringType(), nullable=True), StructField('shipdate', DateType(), nullable=True), StructField('commitdate', DateType(), nullable=True), StructField('receiptdate', DateType(), nullable=True), StructField('shipinstruct', StringType(), nullable=True), StructField('shipmode', StringType(), nullable=True), StructField('comment', StringType(), nullable=True)])


In [7]:
#
# That was pretty busy, let's try that again...
#  loop through the fields of fhe schema and print them out
#

for field in tli.schema.fields:
    print(field.name +" , "+str(field.datatype))

orderkey , type='LongType'
partkey , type='LongType'
suppkey , type='LongType'
linenumber , type='IntegerType'
quantity , type='DoubleType'
extendedprice , type='DoubleType'
discount , type='DoubleType'
tax , type='DoubleType'
returnflag , size=None type='StringType'
linestatus , size=None type='StringType'
shipdate , type='DateType'
commitdate , type='DateType'
receiptdate , type='DateType'
shipinstruct , size=None type='StringType'
shipmode , size=None type='StringType'
comment , size=None type='StringType'


In [8]:
#
# Show the data
#  a is the Dataframe (DF) that we defined early
#  the show() command will list out up to 10 rows
#    pass it an argument for something longer
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.show
#

tli.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"orderkey"  |"partkey"  |"suppkey"  |"linenumber"  |"quantity"  |"extendedprice"  |"discount"  |"tax"  |"returnflag"  |"linestatus"  |"shipdate"  |"commitdate"  |"receiptdate"  |"shipinstruct"    |"shipmode"  |"comment"                                    |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|22498       |1021       |27         |1             |50.0        |46101.0          |0.01        |0.03   |R             |F             |1994-01-15  |1994-02-01    |1994-01-20     |NONE              |FOB         |ely special req

In [9]:
#
# That was pretty busy, let's try that again...
#  use the select method on an existing DF identifying just the columns to keep
#   https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.select

tli_projected = tli.select("orderkey", "linenumber", "quantity", "extendedprice", "linestatus")
tli_projected.show()

---------------------------------------------------------------------------
|"orderkey"  |"linenumber"  |"quantity"  |"extendedprice"  |"linestatus"  |
---------------------------------------------------------------------------
|37504       |1             |15.0        |19656.0          |O             |
|37504       |2             |20.0        |28670.6          |O             |
|37504       |3             |4.0         |7463.84          |O             |
|37504       |4             |32.0        |38185.28         |O             |
|37504       |5             |12.0        |15712.8          |O             |
|37504       |6             |45.0        |60319.35         |O             |
|37505       |1             |47.0        |81725.01         |O             |
|37505       |2             |38.0        |34770.38         |O             |
|37505       |3             |24.0        |39545.76         |O             |
|37505       |4             |27.0        |34813.26         |F             |
------------

In [10]:
#
# Add a simple sort
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.sort
#

tli_projected.sort("orderkey").show()

---------------------------------------------------------------------------
|"orderkey"  |"linenumber"  |"quantity"  |"extendedprice"  |"linestatus"  |
---------------------------------------------------------------------------
|1           |3             |8.0         |12301.04         |O             |
|1           |4             |28.0        |25816.56         |O             |
|1           |5             |24.0        |27389.76         |O             |
|1           |6             |32.0        |33828.8          |O             |
|1           |2             |36.0        |56688.12         |O             |
|1           |1             |17.0        |24710.35         |O             |
|2           |1             |38.0        |36596.28         |O             |
|3           |2             |49.0        |53468.31         |F             |
|3           |1             |45.0        |42436.8          |F             |
|3           |3             |27.0        |32029.56         |F             |
------------

In [11]:
#
# Multiple column sort
#

tli_projected.sort("orderkey", "linenumber").show()

---------------------------------------------------------------------------
|"orderkey"  |"linenumber"  |"quantity"  |"extendedprice"  |"linestatus"  |
---------------------------------------------------------------------------
|1           |1             |17.0        |24710.35         |O             |
|1           |2             |36.0        |56688.12         |O             |
|1           |3             |8.0         |12301.04         |O             |
|1           |4             |28.0        |25816.56         |O             |
|1           |5             |24.0        |27389.76         |O             |
|1           |6             |32.0        |33828.8          |O             |
|2           |1             |38.0        |36596.28         |O             |
|3           |1             |45.0        |42436.8          |F             |
|3           |2             |49.0        |53468.31         |F             |
|3           |3             |27.0        |32029.56         |F             |
------------

In [12]:
#
# Filter some of the data
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.filter
#

# pfs = projected & filtered & sorted
tli_pfs = tli_projected.filter("orderkey <= 5").sort("orderkey", "linenumber")
tli_pfs.show()

---------------------------------------------------------------------------
|"orderkey"  |"linenumber"  |"quantity"  |"extendedprice"  |"linestatus"  |
---------------------------------------------------------------------------
|1           |1             |17.0        |24710.35         |O             |
|1           |2             |36.0        |56688.12         |O             |
|1           |3             |8.0         |12301.04         |O             |
|1           |4             |28.0        |25816.56         |O             |
|1           |5             |24.0        |27389.76         |O             |
|1           |6             |32.0        |33828.8          |O             |
|2           |1             |38.0        |36596.28         |O             |
|3           |1             |45.0        |42436.8          |F             |
|3           |2             |49.0        |53468.31         |F             |
|3           |3             |27.0        |32029.56         |F             |
------------

In [13]:
#
# You can also mix/n/match with SQL and the API
#

session.sql("SELECT orderkey, linenumber, quantity, extendedprice, linestatus \
               FROM tpch.tiny.lineitem") \
     .filter("orderkey <= 5").sort("orderkey", "linenumber").show(51)

---------------------------------------------------------------------------
|"orderkey"  |"linenumber"  |"quantity"  |"extendedprice"  |"linestatus"  |
---------------------------------------------------------------------------
|1           |1             |17.0        |24710.35         |O             |
|1           |2             |36.0        |56688.12         |O             |
|1           |3             |8.0         |12301.04         |O             |
|1           |4             |28.0        |25816.56         |O             |
|1           |5             |24.0        |27389.76         |O             |
|1           |6             |32.0        |33828.8          |O             |
|2           |1             |38.0        |36596.28         |O             |
|3           |1             |45.0        |42436.8          |F             |
|3           |2             |49.0        |53468.31         |F             |
|3           |3             |27.0        |32029.56         |F             |
|3          

In [15]:
#
# You saw that select() was a way to specifically call out the columns you want
#  from an existing DF, but what if there was a bunch of columns and you wanted
#  almost all of them?
#
# The drop() method is the reverse; you identify the columns you'd like to eliminate
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.drop
#

tli.drop("comment", "shipmode", "shipinstruct").show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"orderkey"  |"partkey"  |"suppkey"  |"linenumber"  |"quantity"  |"extendedprice"  |"discount"  |"tax"  |"returnflag"  |"linestatus"  |"shipdate"  |"commitdate"  |"receiptdate"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|22498       |1021       |27         |1             |50.0        |46101.0          |0.01        |0.03   |R             |F             |1994-01-15  |1994-02-01    |1994-01-20     |
|22498       |951        |20         |2             |10.0        |18519.5          |0.06        |0.03   |R             |F             |1993-11-20  |1993-12-23    |1993-12-14     |
|22498       |885        |52         |3             |6.0         |10715.28         |0.07        |0.0

In [16]:
#
# We saw that you can order by multiple columns already.  When you need to have
#  multiple predicates, just chain the filter() methods back to back
#

tli.filter("discount > 0.05") \
   .filter("returnflag = 'A'") \
   .filter("suppkey IN (55, 60, 88)") \
   .filter("shipinstruct LIKE 'TAKE BACK%'") \
   .select("orderkey", "linenumber", "suppkey", "discount", "shipinstruct") \
   .sort("discount", "suppkey", "orderkey", "linenumber").show()

-------------------------------------------------------------------------
|"orderkey"  |"linenumber"  |"suppkey"  |"discount"  |"shipinstruct"    |
-------------------------------------------------------------------------
|15271       |4             |55         |0.06        |TAKE BACK RETURN  |
|23105       |1             |55         |0.06        |TAKE BACK RETURN  |
|32800       |2             |55         |0.06        |TAKE BACK RETURN  |
|39364       |2             |55         |0.06        |TAKE BACK RETURN  |
|42439       |1             |55         |0.06        |TAKE BACK RETURN  |
|53635       |1             |55         |0.06        |TAKE BACK RETURN  |
|3841        |3             |60         |0.06        |TAKE BACK RETURN  |
|25474       |3             |60         |0.06        |TAKE BACK RETURN  |
|53511       |7             |60         |0.06        |TAKE BACK RETURN  |
|6629        |2             |88         |0.06        |TAKE BACK RETURN  |
--------------------------------------

In [17]:
#
# Use standard SQL to see what the shipmode options are
#

session.sql("SELECT DISTINCT(shipmode) FROM tpch.tiny.lineitem").show()

--------------
|"shipmode"  |
--------------
|AIR         |
|MAIL        |
|REG AIR     |
|RAIL        |
|FOB         |
|SHIP        |
|TRUCK       |
--------------



In [18]:
#
# You can get the same thing from the API
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.distinct
#

tli.select("shipmode").distinct().show()

--------------
|"shipmode"  |
--------------
|FOB         |
|SHIP        |
|TRUCK       |
|AIR         |
|MAIL        |
|RAIL        |
|REG AIR     |
--------------



In [19]:
#
# Use standard SQL to find out how many lineitems for each shipmode
#

session.sql(" \
     SELECT shipmode, count() \
       FROM tpch.tiny.lineitem \
      GROUP BY shipmode \
      ORDER BY shipmode").show()

---------------------------
|"shipmode"  |"count(*)"  |
---------------------------
|AIR         |8491        |
|FOB         |8641        |
|MAIL        |8669        |
|RAIL        |8566        |
|REG AIR     |8616        |
|SHIP        |8482        |
|TRUCK       |8710        |
---------------------------



In [20]:
#
# You can get the same thing from the API
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.groupBy
#

tli.group_by("shipmode").count().sort("shipmode").show()

------------------------
|"shipmode"  |"count"  |
------------------------
|AIR         |8491     |
|FOB         |8641     |
|MAIL        |8669     |
|RAIL        |8566     |
|REG AIR     |8616     |
|SHIP        |8482     |
|TRUCK       |8710     |
------------------------



In [21]:
#
# Oh... the DF API almost always has at least 2 ways to perform the same action!
#  here's TWO more ways for this example
#   https://pystarburst.eng.starburstdata.net/dataframe_grouping_functions.html#pystarburst.relational_grouped_dataframe.RelationalGroupedDataFrame
#

tli.group_by("shipmode").agg((col("*"), "count")).sort("shipmode").show()

tli.group_by("shipmode").function("count")("*").sort("shipmode").show()

---------------------------
|"shipmode"  |"count(1)"  |
---------------------------
|AIR         |8491        |
|FOB         |8641        |
|MAIL        |8669        |
|RAIL        |8566        |
|REG AIR     |8616        |
|SHIP        |8482        |
|TRUCK       |8710        |
---------------------------

---------------------------
|"shipmode"  |"count(*)"  |
---------------------------
|AIR         |8491        |
|FOB         |8641        |
|MAIL        |8669        |
|RAIL        |8566        |
|REG AIR     |8616        |
|SHIP        |8482        |
|TRUCK       |8710        |
---------------------------



In [22]:
#
# You can surely calculate multiple aggregate functions for a single group_by
#

tli.group_by("shipmode").agg( \
     (col("shipmode"), "count"), \
     (col("quantity"), "sum"), \
     (col("extendedprice"), "avg"), \
     (col("discount"), "max") \
).sort("count(shipmode)", ascending=False).show()

---------------------------------------------------------------------------------------------
|"shipmode"  |"count(shipmode)"  |"sum(quantity)"  |"avg(extendedprice)"  |"max(discount)"  |
---------------------------------------------------------------------------------------------
|TRUCK       |8710               |223909.0         |35956.15551320322     |0.1              |
|MAIL        |8669               |221528.0         |35827.64891336942     |0.1              |
|FOB         |8641               |219565.0         |35583.13511399151     |0.1              |
|REG AIR     |8616               |219015.0         |35624.070743964665    |0.1              |
|RAIL        |8566               |217810.0         |35615.53778309586     |0.1              |
|AIR         |8491               |216331.0         |35709.31095395133     |0.1              |
|SHIP        |8482               |217969.0         |36043.43757486444     |0.1              |
------------------------------------------------------------

In [23]:
#
# Probably no surprise that this is the equivalent SQL to the last cell
#

session.sql(" \
     SELECT shipmode, count(shipmode), sum(quantity), avg(extendedprice), max(discount) \
       FROM tpch.tiny.lineitem \
      GROUP BY shipmode \
      ORDER BY 2 DESC").show()

---------------------------------------------------------------------------------------------
|"shipmode"  |"count(shipmode)"  |"sum(quantity)"  |"avg(extendedprice)"  |"max(discount)"  |
---------------------------------------------------------------------------------------------
|TRUCK       |8710               |223909.0         |35956.15551320334     |0.1              |
|MAIL        |8669               |221528.0         |35827.64891336941     |0.1              |
|FOB         |8641               |219565.0         |35583.13511399149     |0.1              |
|REG AIR     |8616               |219015.0         |35624.07074396472     |0.1              |
|RAIL        |8566               |217810.0         |35615.53778309599     |0.1              |
|AIR         |8491               |216331.0         |35709.3109539513      |0.1              |
|SHIP        |8482               |217969.0         |36043.437574864336    |0.1              |
------------------------------------------------------------

In [24]:
#
# Show some basic statistics for all columns
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.describe
#

tli.describe().show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"summary"  |"orderkey"          |"partkey"           |"suppkey"          |"linenumber"        |"quantity"          |"extendedprice"    |"discount"            |"tax"                 |"returnflag"  |"linestatus"  |"shipinstruct"    |"shipmode"  |"comment"        |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |60175.0             |60175.0             |60175.0            |60175.0             |60175.0             |60175.0            |60175.0               |60175.0               |60175         |60175     

In [26]:
#
# Yep, that was busy -- let's just look at a few fields
#

tli.describe().select("summary", "quantity", "extendedprice", "discount", "tax").show()

---------------------------------------------------------------------------------------------------
|"summary"  |"quantity"          |"extendedprice"    |"discount"            |"tax"                |
---------------------------------------------------------------------------------------------------
|stddev     |14.406572645801168  |21844.23498496738  |0.03161195042388901   |0.0257735274536384   |
|min        |1.0                 |904.0              |0.0                   |0.0                  |
|max        |50.0                |94949.5            |0.1                   |0.08                 |
|count      |60175.0             |60175.0            |60175.0               |60175.0              |
|mean       |25.527660988782717  |35765.5132608226   |0.049930037390942136  |0.04022451184046523  |
---------------------------------------------------------------------------------------------------



In [27]:
#
# Exercise some of the Trino string functions
#  https://pystarburst.eng.starburstdata.net/dataframe_functions.htm
#

str_test1 = session.sql("SELECT shipmode, shipinstruct FROM tpch.tiny.lineitem") \
     .withColumn("ship_dets", concat_ws(lit(" > "), "shipmode", "shipinstruct")) \
     .withColumn("ship_dets_lc", lower("ship_dets"))
str_test1.show()

str_test2 = session.table("tpch.tiny.lineitem").select("comment") \
     .withColumn("unusual_comment", starts_with("comment", lit("unusual"))) \
     .filter("unusual_comment = true") \
     .withColumn("comment_mod", replace("comment", lit("unusual"), lit("WEIRD")))
str_test2.show()

--------------------------------------------------------------------------------------
|"shipmode"  |"shipinstruct"     |"ship_dets"              |"ship_dets_lc"           |
--------------------------------------------------------------------------------------
|AIR         |COLLECT COD        |AIR > COLLECT COD        |air > collect cod        |
|MAIL        |COLLECT COD        |MAIL > COLLECT COD       |mail > collect cod       |
|REG AIR     |NONE               |REG AIR > NONE           |reg air > none           |
|RAIL        |NONE               |RAIL > NONE              |rail > none              |
|AIR         |COLLECT COD        |AIR > COLLECT COD        |air > collect cod        |
|MAIL        |TAKE BACK RETURN   |MAIL > TAKE BACK RETURN  |mail > take back return  |
|FOB         |NONE               |FOB > NONE               |fob > none               |
|SHIP        |NONE               |SHIP > NONE              |ship > none              |
|REG AIR     |NONE               |REG AIR >

In [28]:
#
# Let's join some tables
#  https://pystarburst.eng.starburstdata.net/dataframe.html#pystarburst.dataframe.DataFrame.join
#

ordersDF = session.table("tpch.tiny.orders")
lineitemDF = session.table("tpch.tiny.lineitem").rename("orderkey", "li_ok")

joinedDF = lineitemDF.join(ordersDF, ordersDF.orderkey == lineitemDF.li_ok) \
     .select("orderkey", "linenumber", "extendedprice", "linestatus", "custkey") \
     .sort("orderkey", "linenumber")
mydf =joinedDF.to_pandas()
print(mydf);

       orderkey  linenumber  extendedprice linestatus  custkey
0             1           1       24710.35          O      370
1             1           2       56688.12          O      370
2             1           3       12301.04          O      370
3             1           4       25816.56          O      370
4             1           5       27389.76          O      370
...         ...         ...            ...        ...      ...
60170     60000           2       40131.32          O     1426
60171     60000           3       43112.25          F     1426
60172     60000           4       33966.83          O     1426
60173     60000           5       46052.98          O     1426
60174     60000           6       78157.35          O     1426

[60175 rows x 5 columns]


In [29]:
#
# Let's join 4 tables together and determine the average lineitem price by nation name
#  note: renaming the (logical) FK col names to aid in auto-renaming confusion that occurs
#

smaller_orders_lineitems = joinedDF.drop("linenumber", "linestatus") \
     .rename("custkey", "sol_ck").filter("orderkey BETWEEN 100 AND 199")

customerDF = session.sql("SELECT custkey, nationkey AS c_nk FROM tpch.tiny.customer")

o_li_c = smaller_orders_lineitems.join(customerDF, \
                smaller_orders_lineitems.sol_ck == customerDF.custkey)

nationDF = session.table("tpch.tiny.nation").drop("regionkey").drop("comment")

nation_avg_price = o_li_c.join(nationDF, o_li_c.c_nk == nationDF.nationkey) \
     .rename("name", "nation_name") \
     .select("nation_name", "extendedprice") \
     .group_by("nation_name").avg("extendedprice") \
     .with_column("avg_price", round("avg(extendedprice)", lit(2))) \
     .select("nation_name", "avg_price") \
     .sort("avg_price", ascending=False)
nation_avg_price.show()

--------------------------------
|"nation_name"   |"avg_price"  |
--------------------------------
|SAUDI ARABIA    |54401.73     |
|VIETNAM         |48446.24     |
|ROMANIA         |40023.37     |
|PERU            |38430.95     |
|UNITED KINGDOM  |37834.35     |
|FRANCE          |36963.62     |
|IRAN            |36862.28     |
|ETHIOPIA        |36224.73     |
|GERMANY         |35429.85     |
|JORDAN          |34802.85     |
--------------------------------



In [30]:
#
# SQL version of the above cell, plus verifying the results are identical
#  by showing the except() output is empty
#

nation_avg_price_sql = session.sql(" \
     SELECT n.name AS nation_name, \
            ROUND(AVG(li.extendedprice), 2) AS avg_price \
       FROM tpch.tiny.lineitem li \
       JOIN tpch.tiny.orders o   ON (li.orderkey = o.orderkey) \
       JOIN tpch.tiny.customer c ON (o.custkey = c.custkey) \
       JOIN tpch.tiny.nation n   ON (c.nationkey = n.nationkey) \
      WHERE o.orderkey BETWEEN 100 and 199 \
      GROUP BY n.name \
      ORDER BY avg_price DESC")
nation_avg_price_sql.show()


nation_avg_price.except_(nation_avg_price_sql).show()

--------------------------------
|"nation_name"   |"avg_price"  |
--------------------------------
|SAUDI ARABIA    |54401.73     |
|VIETNAM         |48446.24     |
|ROMANIA         |40023.37     |
|PERU            |38430.95     |
|UNITED KINGDOM  |37834.35     |
|FRANCE          |36963.62     |
|IRAN            |36862.28     |
|ETHIOPIA        |36224.73     |
|GERMANY         |35429.85     |
|JORDAN          |34802.85     |
--------------------------------

-------------------------------
|"nation_name"  |"avg_price"  |
-------------------------------
|               |             |
-------------------------------



## You definitely have some **optionality** with the DataFrame API.

## That's enough examples for this notebook :)